## Coletando os dados


In [33]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
# lista do caminho dos arquivos em formato csv
arquivos_csv = [
    r'D:\Projetos\Analise_dados_PRF_acidentes\files\acidentes2020.csv',
    r'D:\Projetos\Analise_dados_PRF_acidentes\files\acidentes2021.csv', 
    r'D:\Projetos\Analise_dados_PRF_acidentes\files\acidentes2022.csv',
    r'D:\Projetos\Analise_dados_PRF_acidentes\files\acidentes2023.csv'
]

In [35]:
# lista de arquivos armazenados
dfs = []

#lê cada arquivo e inclui na lista dfs
for arquivo in arquivos_csv:
    df = pd.read_csv(arquivo, delimiter=';', encoding='ISO-8859-1')
    dfs.append(df)



In [36]:
# concatena os arquivos da lista dfs
df_final = pd.concat(dfs, axis=0, ignore_index=True)

In [37]:
df_final.shape

(576136, 35)

### Criação do Metadados

In [38]:
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100 ,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='tipo')
    metadata = metadata.reset_index(drop=True)# reseta o indice do DataFrame e apaga o índice antigo

    return metadata

In [39]:
generate_metadata(df_final)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,feridos_leves,int64,0,0.00,2
1,ilesos,int64,0,0.00,2
2,feridos_graves,int64,0,0.00,2
3,mortos,int64,0,0.00,2
4,id_veiculo,int64,0,0.00,400417
5,br,float64,1726,0.30,122
6,pesid,float64,4,0.00,576132
7,idade,float64,57598,10.00,173
8,ano_fabricacao_veiculo,float64,38930,6.76,70
9,id,float64,0,0.00,242355


### Filtrando os dados para o bojetivo do projeto


In [40]:
# criando um novo df com o filtro de apenas pessoas que morreram
df_obt = df_final.loc[(df_final['mortos']==1)]

In [41]:
df_obt.shape

(20249, 35)

In [42]:
generate_metadata(df_obt)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,feridos_leves,int64,0,0.00,1
1,ilesos,int64,0,0.00,1
2,feridos_graves,int64,0,0.00,1
3,mortos,int64,0,0.00,1
4,id_veiculo,int64,0,0.00,17803
5,br,float64,15,0.07,110
6,pesid,float64,0,0.00,20249
7,idade,float64,1106,5.46,105
8,ano_fabricacao_veiculo,float64,2255,11.14,64
9,id,float64,0,0.00,17421


## Tratamento dos dados

Nesse fase será feita a limpeza dos dados para iniciar a análise descritiva

### Eliminação de colunas 

Após o filtro na coluna "mortos" podemos eliminar algumas colunas que ficaram redundantes e também algumas que a princípio não serão utilizadas na análise

In [43]:
colunas_drop = ['feridos_leves', 'ilesos', 'feridos_graves', 'mortos',
                'estado_fisico', 'classificacao_acidente', 'marca',
                'ano_fabricacao_veiculo', 'uop', 'delegacia' ]

In [44]:
# removendo a lista de colunas selecionadas
df_obt = df_obt.drop(columns=colunas_drop)

In [45]:
df_obt.shape

(20249, 25)

In [46]:
generate_metadata(df_obt)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,id_veiculo,int64,0,0.00,17803
1,id,float64,0,0.00,17421
2,pesid,float64,0,0.00,20249
3,idade,float64,1106,5.46,105
4,br,float64,15,0.07,110
5,latitude,object,0,0.00,15774
6,sexo,object,0,0.00,3
7,tipo_envolvido,object,0,0.00,4
8,tipo_veiculo,object,0,0.00,22
9,uso_solo,object,0,0.00,2


In [51]:
df_obt.describe()

,id,pesid,data_inversa,horario,id_veiculo,idade
count,20249.000000,2.024900e+04,20249,20249,20249.000000,19143.000000
mean,407010.229592,8.978628e+05,2021-11-27 19:04:56.725764352,2023-11-01 13:31:08.619684864,724202.078671,41.467638
min,260036.000000,5.786340e+05,2020-01-01 00:00:00,2023-11-01 00:00:00,464131.000000,0.000000
25%,330222.000000,7.345680e+05,2020-12-20 00:00:00,2023-11-01 07:15:00,592578.000000,29.000000
50%,400123.000000,8.944870e+05,2021-11-28 00:00:00,2023-11-01 15:00:00,725088.000000,40.000000
75%,485215.000000,1.054379e+06,2022-10-29 00:00:00,2023-11-01 19:20:00,851496.000000,52.000000
max,554938.000000,1.226987e+06,2023-09-30 00:00:00,2023-11-01 23:59:00,985828.000000,1947.000000
std,86501.764324,1.844939e+05,NaN,NaN,148794.818636,25.603431


### Análise dos tipos de dados

In [48]:
# Alterando a coluna data_inversa para o tipo datetime
df_obt['data_inversa'] = pd.to_datetime(df_obt['data_inversa'])

In [49]:
# Alterando a coluna horario para o tipo datetime
df_obt['horario'] = pd.to_datetime(df_obt['horario'])

C:\Users\sv_ya\AppData\Local\Temp\ipykernel_22044\2701647929.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_obt['horario'] = pd.to_datetime(df_obt['horario'])


In [50]:
# Alterando a coluna br para o tipo object
df_obt['br'] = df_obt['br'].astype(object)

### Análise dos valores Nulos

In [29]:
# Como a diferença não é significativa para a variável idade, os valores nulos serão preenchidos com a média.
# Nas colunas 'km' e 'br' os valores nulos serão preenchidos com 'Não informado'
replace = {
    'br' : 'Desconhecido',
    'km' : 'Desconhecido',
    'idade' : df_obt['idade'].mean()
}
df_obt.fillna(value=replace, inplace=True)

In [32]:
df_obt.to_csv('teste.csv', index=False)